### Training Loop

- the goal of callbacks in FastAI is to **allow the training loop to be highly customizable** 
- this is accomplished by injecting conditions such as `on_batch_begin`, `on_batch_end`, `on_epoch_begin`, `on_epoch_end` etc that trigger corresponding custom callback functions

#### Basic Mini-Batch Training Loop

In [ ]:
for epoch in range(epochs):
    learn.model.train() # set to training mode
    for xb,yb in learn.data.train_dl: # grabs mini-batch from dataloader
        loss = learn.loss_func(learn.model(xb), yb) # compute loss
        loss.backward()  # compute gradients
        learn.opt.step() # update parameters
        learn.opt.zero_grad() # reset gradients

#### The "infinitely customizable" training loop

The code snippet below demonstrates the scafold of a training loop that allows a wide variety of customizability based on callbacks.

In [ ]:
callbacks.on_train_begin()
for _ in range(epoch):
    callbacks.on_epoch_begin()  #---------------callback before an epoch begins
    learn.model.train()
    for xb,yb in learn.data.train_dl:
        callbacks.on_batch_begin() #------------callback before a batch begins
        y = model(xb)
        callbacks.on_loss_begin()  #------------callback before loss computation begins
        loss = loss_func(y, yb)
        callbacks.on_backward_begin() #---------callback before gradient computation begin
        loss.backward() # backward pass
        callbacks.on_backward_end()  #----------callback after gradient computation begin
        opt.step()
        callbacks.on_step_end() #---------------callback after param update begins
        opt.zero_grad() 
        callbacks.on_batch_end() #--------------callback after a batch
    callbacks.on_epoch_end() #------------------callback after an epoch
callbacks.on_train_end() #----------------------callback after training

### Callback class

- to provide the generic functionality of all the callbacks listed in the training loop above, it is convenient to define a `Callback` class template.
- all existing and new custom callbacks to be implemented need to inherit from this base class

The toy-version of the `Callback` class is provided below, for detail of the actual `Callback` class, see: https://docs.fast.ai/callback.html#Callback

In [ ]:
class Callback():
    def begin_fit(self, learn):
        self.learn = learn
        return True
    def after_fit(self): return True
    def begin_epoch(self, epoch):
        self.epoch=epoch
        return True
    def begin_validate(self): return True
    def after_epoch(self): return True
    def begin_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        return True
    def after_loss(self, loss):
        self.loss = loss
        return True
    def after_backward(self): return True
    def after_step(self): return True

Closely associated with the `Callback` class is the `CallbackHandler` class, which combines all the callbacks together and call any relevant callback functions for each training stage. See: https://docs.fast.ai/callback.html#CallbackHandler

In [ ]:
class CallbackHandler():
    def __init__(self,cbs=None):
        self.cbs = cbs if cbs else []

    def begin_fit(self, learn):
        self.learn,self.in_train = learn,True
        learn.stop = False
        res = True
        for cb in self.cbs: res = res and cb.begin_fit(learn)
        return res

    def after_fit(self):
        res = not self.in_train
        for cb in self.cbs: res = res and cb.after_fit()
        return res
    
    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train=True
        res = True
        for cb in self.cbs: res = res and cb.begin_epoch(epoch)
        return res

    def begin_validate(self):
        self.learn.model.eval()
        self.in_train=False
        res = True
        for cb in self.cbs: res = res and cb.begin_validate()
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs: res = res and cb.after_epoch()
        return res
    
    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs: res = res and cb.begin_batch(xb, yb)
        return res

    def after_loss(self, loss):
        res = self.in_train
        for cb in self.cbs: res = res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res = True
        for cb in self.cbs: res = res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs: res = res and cb.after_step()
        return res
    
    def do_stop(self):
        try:     return self.learn.stop
        finally: self.learn.stop = False

This is roughly how FastAI v1.0 implements it. But looking at the code, there's still quite a bit of repetition with all the:

`res = condition
for cb in self.cbs: res = res and cb._method_()
return res`

The new `Runner` class appears to be how future versions of FastAI will be implemented at the time of writing.